In [1]:
import requests
import re
from bs4 import BeautifulSoup
import openpyxl
from string import digits

res = requests.get('https://ch.cauhs.or.kr/medical/medical.asp?cat_no=02010000') # URL
soup = BeautifulSoup(res.text, 'html.parser')
doc_title = soup.select_one('div.page')
doc_title1 = doc_title.select('li a')
dept_link = []
for i in doc_title1 :
    link = 'https://ch.cauhs.or.kr/medical/' + i['href']
    dept_link.append(link)
doctor_link = []
for i in dept_link :
    res = requests.get(i)
    soup = BeautifulSoup(res.text, 'html.parser')
    real_link = soup.select('div.doctor_img a')
    for i in real_link :  
        i = i['href']
        num = re.findall('\d+', i)
        StrA = "".join(num)
        link = 'https://ch.cauhs.or.kr/medical/doc_detail.aspx?empno=' + StrA
        doctor_link.append(link)
doc_id = 4262
for doctor in doctor_link :
    res = requests.get(doctor)
    soup = BeautifulSoup(res.text, 'html.parser')   
    wb = openpyxl.load_workbook('중앙대학교병원_학력.xlsx') # Chung-Ang_University_Degree
    ws = wb.worksheets[0]
    doc_id += 1
    table_num = soup.select('ul.bul_ty1')
    yr_start = []
    yr_end = []
    docter_content = []
    if len(table_num) == 7 or len(table_num) == 6 or len(table_num) == 5 or len(table_num) == 4 or len(table_num) == 3 or len(table_num) == 2: 
        categorys1 = soup.select('ul h3')[1].text.strip()
        categorys2 = soup.select('ul h3')[0].text.strip()
        category_1 = categorys1.find('학력') # Degree
        category_2 = categorys2.find('학력') # Degree
        if category_1 == 0 : 
            ul_list = soup.select('ul.bul_ty1')[1]
            li = ul_list.select('li')
            for i in li : 
                zx = i.text.strip()
                table = str.maketrans('', '', digits)
                title = zx.translate(table).replace(" : ","").replace('.. ','').replace(' ', '').replace('.','').replace('-','').replace('~','')
                num = re.findall(r'\d+', zx)
                name = soup.select_one('p.doctor_name').text.strip()
                dept = soup.select_one('p.doctor_part').text.strip()
                if title == None :
                    docter_content.append('')
                if title != None : 
                    docter_content.append(title)
                if num == None :
                    yr_start.append('')
                    yr_end.append('')
                elif num != None:
                    if len(num) == 6 :
                        date = ".".join(num[0:3]) 
                        yr_start.append(date)
                        date1 = ".".join(num[3:6]) 
                        yr_end.append(date1)  
                    elif len(num) == 4 :
                        date = ".".join(num[0:2]) 
                        yr_start.append(date)
                        date1 = ".".join(num[2:4]) 
                        yr_end.append(date1)
                    elif len(num) == 3 : 
                        yr_start.append('') 
                        yr_end.append('')
                    elif len(num) == 2 :
                        date = ".".join(num[0:2])
                        if len(date) == 6 :
                            yr_start.append(date)
                            yr_end.append(date)
                        if len(date) == 7 :              
                            yr_start.append(date)
                            yr_end.append(date)
                        if len(date) == 9 :
                            date = date.split('.')
                            yr_start.append(date[0])
                            yr_end.append(date[1])
                    elif len(num) == 1 :
                        yr_start.append(num[0])
                        yr_end.append(num[0])
                    elif len(num) == 0 :
                        yr_start.append('')
                        yr_end.append('')
            for qe in range(len(docter_content)):
                real_data = []
                real_data.append('doc0000' + str(doc_id))
                real_data.append(yr_start[qe])
                real_data.append(yr_end[qe])
                real_data.append(docter_content[qe])
                ws.append(real_data)
                print(real_data)  
        elif category_2 == 0 :
            ul_list = soup.select('ul.bul_ty1')[0]
            li = ul_list.select('li')
            for i in li : 
                zx = i.text.strip()
                table = str.maketrans('', '', digits)
                title = zx.translate(table).replace(':','').replace(" : ","").replace('.. ','').replace(' ', '').replace('.','').replace('-','').replace('~','')
                num = re.findall(r'\d+', zx)
                if title == None :
                    docter_content.append('')
                if title != None : 
                    docter_content.append(title)
                if num == None :
                    yr_start.append('')
                    yr_end.append('')
                elif num != None:
                    if len(num) == 6 :
                        date = ".".join(num[0:3]) 
                        yr_start.append(date)
                        date1 = ".".join(num[3:6]) 
                        yr_end.append(date1)                      
                    elif len(num) == 4 :
                        date = ".".join(num[0:2]) 
                        yr_start.append(date)
                        date1 = ".".join(num[2:4]) 
                        yr_end.append(date1)
                    elif len(num) == 3 : 
                        yr_start.append('')
                        yr_end.append('')
                    elif len(num) == 2 :
                        date = ".".join(num[0:2])
                        if len(date) == 6 :
                            yr_start.append(date)
                            yr_end.append(date)
                        if len(date) == 7 :              
                            yr_start.append(date)
                            yr_end.append(date)
                        if len(date) == 9 :
                            date = date.split('.')
                            yr_start.append(date[0])
                            yr_end.append(date[1])
                    elif len(num) == 1 :
                        yr_start.append(num[0])
                        yr_end.append(num[0])
                    elif len(num) == 0 :
                        yr_start.append('')
                        yr_end.append('')        
            for qe in range(len(docter_content)):
                real_data = []
                real_data.append('doc0000' + str(doc_id))
                real_data.append(yr_start[qe])
                real_data.append(yr_end[qe])
                real_data.append(docter_content[qe])
                ws.append(real_data)
                print(real_data)     
    wb.save('중앙대학교병원_학력.xlsx') # Chung-Ang_University_Degree
    wb.close()

['doc00004263', '1990', '1990', '중앙대학교의과대학졸업']
['doc00004263', '1996', '1996', '중앙대학교의과대학원석사']
['doc00004263', '1999', '1999', '중앙대학교의과대학원박사']
['doc00004264', '1996.03', '2001.02', '연세대학교의과대학']
['doc00004264', '2003.09', '2007.08', '연세대학교의과대학원석사(가정의학전공)']
['doc00004264', '2007.09', '2007.09', '현재연세대학교의과대학원박사과정중(가정의학전공)']
['doc00004265', '2007', '2007', '중앙대학교약학대학졸업']
['doc00004265', '2013', '2013', '중앙대학교의학전문대학원석사']
['doc00004265', '2020', '2020', '중앙대학교의과대학대학원박사(의학박사)']
['doc00004266', '1997', '1997', '중앙대학교의과대학학사']
['doc00004266', '2004', '2004', '중앙대학교의학박사']
['doc00004266', '2002', '2002', '내과전문의취득']
['doc00004266', '2004', '2004', '감염내과분과전문의취득']
['doc00004267', '1999', '1999', '경북대학교의과대학졸업']
['doc00004267', '2007', '2007', '내과전문의취득']
['doc00004267', '2008', '2008', '울산대학교의학석사']
['doc00004267', '2009', '2009', '감염내과전문의취득']
['doc00004267', '2014', '2014', '울산대학교의학박사']
['doc00004268', '2007.02', '2007.02', '연세대학교의학과학사']
['doc00004268', '2016.02', '2016.02', '울산대학교대학원석사']
['doc00004268

['doc00004323', '2011', '2011', '중앙대학교의과대학의학부졸업']
['doc00004323', '2014', '2014', '중앙대학교의과대학원의학석사(성형외과)']
['doc00004323', '2019', '2019', '중앙대학교의과대학원의학박사(성형외과)']
['doc00004324', '1976.3', '1982.2', '중앙대학교의과대학학사']
['doc00004324', '1983.3', '1985.2', '중앙대학교의과대학석사']
['doc00004324', '1985.3', '1988.8', '중앙대학교의과대학박사']
['doc00004324', '1982.3', '1986.2', '중앙대병원인턴및소아과전공의']
['doc00004324', '1990', '1990', '서울대어린이병원소아신장학전임의']
['doc00004324', '1997', '1998', '미국UCLA소아신장fellow']
['doc00004325', '1978.3.3', '1984.2.24', '중앙대학교의과대학의학사(의학과)']
['doc00004325', '1986.3.4', '1988.2.23', '중앙대학교대학원의학석사(소아과학전공)']
['doc00004325', '1988.3.3', '1991.2.22', '중앙대학교대학원의학박사(소아과학전공)']
['doc00004326', '', '', '중앙대학교의과대학학사']
['doc00004326', '', '', '중앙대학교의과대학의학석사']
['doc00004326', '', '', '중앙대학교의과대학의학박사']
['doc00004327', '1999', '2005', '중앙대학교의과대학의학과학사']
['doc00004327', '2007', '2009', '중앙대학교대학원의학석사']
['doc00004327', '2015.2', '2015.2', '중앙대학교대학원의학박사']
['doc00004328', '1998.03', '2004.02', '중앙대학교의과대학졸업학사']
['doc0000

['doc00004387', '1997', '1997', '인하대학교의학학사']
['doc00004387', '2008', '2008', '강원대학교의학석사']
['doc00004387', '2011', '2011', '성균관대학교의학박사']
['doc00004388', '2003', '2003', '이화여자대학교의과대학']
['doc00004388', '2007', '2007', '이화여자대학교의과대학석사']
['doc00004388', '2012', '2012', '이화여자대학교의과대학박사']
['doc00004389', '2004', '2004', '서울대의대졸업']
['doc00004389', '2011', '2011', '서울대의학석사']
['doc00004390', '2006', '2006', '순천향의대졸']
['doc00004390', '2010', '2010', '순천향대의학석사']
['doc00004390', '2017', '2017', '카톨릭대학교의학박사']
['doc00004391', '2006', '2006', '서울대학교의과대학']
['doc00004391', '2011', '2011', '서울대학교의과대학석사']
['doc00004391', '2013', '2013', '서울대학교의과대학박사']
['doc00004392', '2007', '2007', '순천향대학교의학과졸업']
['doc00004393', '2010', '2010', '연세대학교의과대학졸업']
['doc00004393', '2014', '2014', '성균관대학교의학석사']
['doc00004394', '2005.02', '2005.02', '건국대학교수의학과']
['doc00004394', '2010.02', '2010.02', '경희의학전문대학원의학석사']
['doc00004394', '2015.02', '2015.02', '울산대학교의학박사']
['doc00004395', '2008', '2008', '중앙대학교의과대학의학사졸업']
['doc00004395',

['doc00004448', '1997', '1997', '중앙대학교의과대학졸업']
['doc00004448', '2002', '2002', '중앙대학교의학과석사']
['doc00004448', '2011', '2011', '울산대학교의학박사']
['doc00004449', '2005', '2005', '중앙대학교의과대학졸업']
['doc00004449', '2009', '2009', '중앙대학교의학석사']
['doc00004449', '2016', '2016', '성균관대학교의학박사과정중']
['doc00004450', '2008', '2008', '중앙대학교의과대학졸업(학사)']
['doc00004450', '2015', '2015', '중앙대학교의과대학원졸업(석,박사통합)']
['doc00004450', '2020', '2020', '한국방송통신대학교통계학과졸업(석사)']
['doc00004451', '2009', '2009', '중앙대학교의과대학의학사졸업']
['doc00004451', '2014', '2014', '중앙대학교의과대학의학석사졸업']
['doc00004452', '2011', '2011', '중앙대학교의과대학졸업']
['doc00004452', '2016', '2016', '울산대학교정형외과학의학석사']
['doc00004452', '2020', '2020', '울산대학교정형외과학의학박사']
['doc00004453', '2010.02', '2010.02', '서울대학교의과대학졸업']
['doc00004454', '2011', '2011', '중앙대학교의과대학졸업(학사)']
['doc00004454', '2015', '2015', '중앙대학교의과대학원졸업(석사)']
['doc00004455', '1990', '1990', ':중앙대학교의과대학의학과졸업']
['doc00004455', '1995', '1995', ':중앙대학교대학원석사(진단검사의학)']
['doc00004455', '1999', '1999', ':중앙대학교대학원박사(진단검사

['doc00004509', '2014.02', '2014.02', '중앙대학교의과대학졸업(학사)']
['doc00004509', '2018.03', '2020.02', '서울대학교의과대학내과학(석사)']
['doc00004509', '2020.02', '2020.02', '현재서울대학교의과대학내과학재학중(박사)']
['doc00004511', '2013', '2013', '중앙대학교의과대학졸업(학사)']
['doc00004511', '2017', '2017', '？앙대학교의과대학원졸업(석사)']
['doc00004512', '2017', '2017', '년중앙대학교의학전문대학원졸업']
['doc00004514', '1982', '1982', '서울대학교의과대학졸업']
['doc00004514', '1991', '1991', '서울대학교대학원석사']
['doc00004514', '1995', '1995', '서울대학교대학원박사']
['doc00004515', '1986', '1986', '중앙대학교의과대학졸업']
['doc00004515', '1989', '1989', '중앙대학교의과대학석사(내과학)']
['doc00004515', '1996', '1996', '중앙대학교의과대학박사(내과학)']
['doc00004516', '', '', '중앙대학교의과대학박사']
['doc00004517', '', '', '중앙대학교의과대학의학박사']
['doc00004518', '', '', '중앙대학교의과대학학사,석사,박사']
['doc00004519', '2003', '2003', '고려대학교의과대학졸업']
['doc00004519', '2007', '2007', '고려대학교의학석사']
['doc00004519', '2013', '2013', '고려대학교의학박사']
['doc00004520', '', '', '연세대의과대학졸']
['doc00004520', '', '', '연세대의학석사']
['doc00004520', '', '', '연세대의학박사']
['doc00004

['doc00004573', '2013', '2013', '이화여자대학교의학전문대학원석사']
['doc00004573', '2018', '2018', '이화여자대학교대학원박사과정중']
['doc00004574', '', '', '동아대학교의과대학졸업']
['doc00004575', '2014', '2014', '년중앙대학교의과대학졸업(학사)']
['doc00004575', '2021', '2021', '년서울대학교의과대학대학원수료중(석사)']
['doc00004576', '1986', '1986', '중앙대학교의과대학졸']
['doc00004576', '1989', '1989', '중앙대학교의과대학원석사(예방의학)']
['doc00004576', '1993', '1993', '중앙대학교의과대학원박사(예방의학)']
['doc00004577', '1994', '1994', '중앙대학교의과대학졸업(학사)']
['doc00004577', '1998', '1998', '중앙대학교대학원줄업(석사)']
['doc00004577', '2007', '2007', '중앙대학교대학원졸업(박사)']
['doc00004578', '1997.02', '1997.02', '중앙대학교의과대학졸업(학사)']
['doc00004578', '2000.08', '2000.08', '중앙대학교대학원의학과졸업(석사)']
['doc00004578', '2003.08', '2003.08', '중앙대학교대학원의학과졸업(박사)']
['doc00004579', '2007', '2007', '중앙대학교의학부졸업']
['doc00004579', '2014', '2014', '중앙대학교대학원의학과정신과학박사']
['doc00004580', '2014.02', '2014.02', '중앙대학교의과대학졸업(학사)']
['doc00004580', '2018.08', '2018.08', '중앙대학교대학원졸업(석사)']
['doc00004581', '2015.02', '2015.02', '중앙대학교의과대학졸업(학사)']
[

['doc00004635', '2007', '2007', '순천향대학교의과대학학사']
['doc00004635', '2012', '2012', '순천향대학교대학원석사(영상의학과)']
['doc00004635', '2017', '2017', '순천향대학교대학원박사(영상의학과)']
['doc00004636', '1986.02', '1986.02', '중앙대학교의과대학졸업']
['doc00004636', '1994.02', '1994.02', '중앙대학교용산병원신경외과전공의수료및전문의취득']
['doc00004636', '1997.02', '1997.02', '중앙대학교대학원박사학위취득']
['doc00004636', '2001', '2002', '미국ClevelandClinic연수']
['doc00004637', '1988', '1988', '중앙대학교의과대학졸업']
['doc00004637', '1991', '1991', '의학석사(중앙대)']
['doc00004637', '1993', '1993', '신경외과전문의(중앙대학교수련)']
['doc00004637', '1999', '1999', '의학박사(중앙대)']
['doc00004638', '1998', '1998', '중앙대학교의과대학졸업']
['doc00004638', '2005', '2005', '중앙대학교의학석사']
['doc00004638', '2012', '2012', '중앙대학교의학박사']
['doc00004639', '1996', '1996', '연세대원주의대졸']
['doc00004639', '2003', '2003', '연세대의학석사']
['doc00004639', '2006', '2006', '연세대의학박사']
['doc00004640', '2007', '2007', '서울대학교의과대학의학사졸업']
['doc00004640', '2013', '2013', '서울대학교의과대학의학석사졸업']
['doc00004641', '2009.02', '2009.02', '중앙대학교의과대학(학사)']
['

['doc00004700', '2001', '2001', '중앙대학교의과대학졸업']
['doc00004700', '2005', '2005', '중앙대학교대학원의학과석사학위취득']
['doc00004700', '2010', '2010', '중앙대학교대학원의학과박사학위취득']
['doc00004701', '2001', '2001', '년중앙대학교의과대학의학과졸업(의학사)']
['doc00004701', '2008', '2008', '년연세대학교대학원의학과졸업(의학석사)']
['doc00004701', '2016', '2016', '년중앙대학교대학원의학과졸업(의학박사)']
['doc00004701', '', '', 'Thesis:Developmentofnewstrategiesfortheonehandedchestcompressiontechniqueusedinpediatriccardiopulmonaryresuscitation']
['doc00004701', '', '', '의사면허취득:']
['doc00004701', '', '', '전문의자격증(응급의학과전문의)취득:']
['doc00004704', '2010.02', '2010.02', '한양대학교의과대학의학과졸업(의학사)']
['doc00004704', '2015.02', '2015.02', '강원대학교대학원의학과(응급의학)졸업(의학석사)']
['doc00004704', '2017.02', '2017.02', '한양대학교대학원의학과(의공학)수료(박사과정)']
['doc00004705', '2011', '2011', '연세대학교원주의과대학졸업(학사)']
['doc00004705', '2020', '2020', '중앙대학교대학원의학과석사과정']
['doc00004706', '2016', '2016', '중앙대학교의학전문대학원졸업(석사)']
